# Mirko Michele D'Angelo - Assignment 3

First we load the data of the MNIST dataset, both training and test sets.

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import idx2numpy
#load training images
tr_images=idx2numpy.convert_from_file('./dataset/train-images.idx3-ubyte')
tr_labels=idx2numpy.convert_from_file('./dataset/train-labels.idx1-ubyte')
#load test images
ts_images=idx2numpy.convert_from_file('./dataset/t10k-images.idx3-ubyte')
ts_labels=idx2numpy.convert_from_file('./dataset/t10k-labels.idx1-ubyte')

# Implementation
First we implement our RBM with the CD-1 training algorithm, the implementation is in the RBM class with the following methods:
- The __ __init__ __ constructor wll just initialize the values used for the biases and weights.
- __sample_hidden__ implement for the corrisponding operation of sampling hidden units from the visible ones.
- __sample_visible__ implement for the corrisponding operation of sampling hidden visible from the hidden ones.
- the __train__ method implements the actual training using the CD-1 algorithm with minibatch and MSE to monitor the reconstruction error.
- the __encode__ method allows us to get the activations of hidden units and use them to encode data.
- the __reconstruct__ method allows us to reconstsruct data.

Also a simple sigmoid implementation and a utility sampling method are used to implement the other methods in the class.
## parameters initialization
Parameters are initialized according to [this paper](http://www.cs.toronto.edu/%7Ehinton/absps/guideTR.pdf) :

Weights are initialized with a gaussian distribution $N(0,0.01)$, the hidden biases with a value of 0. The visible biases are all initialized with value $\log(p_i/(1-p_i))$ where $p_i$ is is the proportion of training vectors in which unit $i$ is on, also note that since $p_i$ might become positive infinite or negative infinite due to precision erros in the fraction the values that go to infinite are clipped to have the maximum or minimum finite value available depending on their sign. 

In [12]:
class RBM:
    def __init__(self,visible_size,hidden_size):
        self.visible_bias= np.zeros(visible_size,dtype='longdouble')
        self.hidden_bias= np.zeros(hidden_size,dtype='longdouble')

        self.weights=np.random.normal(scale=0.01,size=(visible_size,hidden_size))
        print(f"buildinig a RBM with {visible_size} visible units and {hidden_size} hidden units")
    def _sigmoid(self,x):
        return 1/(1+np.exp(-x))
    def _sample(self,prob):
        return (prob > np.random.rand(*prob.shape)).astype(np.longdouble)
    def sample_hidden(self,v):
        ha_prob= self._sigmoid(v@self.weights+self.hidden_bias)
        ha_states= self._sample(ha_prob)
        return ha_prob,ha_states
    def sample_visible(self,h):
        recon_prob= self._sigmoid(h@self.weights.T+self.visible_bias)
        recon_act= self._sample(recon_prob)
        return recon_prob,recon_act

    def train(self,values,eta=0.01,epochs=100,batch_size=64):
        print(f"training over {values.shape[0]} samples with {values.shape[1]} features \nepochs={epochs}\t batch size={batch_size}\t learning rate={eta}")
        for e in range(epochs):
            for i in range(0,values.shape[0],batch_size):
                # clamp data as input
                clamped_data= values[i:i+batch_size]
                #sample h given v
                ha_prob,ha_states=self.sample_hidden(clamped_data)
                #calculate wake part
                wake=clamped_data.T@ha_prob
                #sample v given h
                recon_prob,recon_act=self.sample_visible(ha_states)
                active_prob=self._sigmoid(recon_act@self.weights+ self.hidden_bias)
                #calculate dream part
                dream=recon_act.T@active_prob
                delta_w=(wake-dream)/batch_size
                delta_bh = (np.mean(ha_prob-active_prob, axis=0))
                delta_bv = (np.mean(clamped_data-recon_act, axis=0))

                self.weights+=eta*delta_w
                self.hidden_bias+=eta*delta_bh
                self.visible_bias+=eta*delta_bv
            clamped_data= values
            recon_act= self.reconstruct(clamped_data)
            print(f"epoch no.{e+1} reconstruction error: {np.mean((clamped_data-recon_act)**2)}")
    def reconstruct(self,data):
        _,ha_states=self.sample_hidden(data)
        recon_prob,recon_act=self.sample_visible(ha_states)
        return recon_act
    def encode(self,data):
        #sample h given v
        _,ha_states=self.sample_hidden(data)
        print(f"{ha_states.shape[0]} samples encoded with {ha_states.shape[1]} hidden units")
        return ha_states

## RBM training
Now we train the RBM, first the data from mnist dataset is flattened from a $(28 \times 28)$ matrix of integers between 0 and 255 to an array of 768 integers.
After the flattening binarization is applied with a random threshold generated by a uniform distribution. The reason for the flattening is just to be able to feed it into the rbm while the binarization is useful since it allows the contrastive divergence algorithm to properly work.

The training is done with a small grid search over hidden units and learning rates since changing batch sizes, at least during my tests, doesn't affect a lot the final results. The dataset has been split using a holdout with 80% training set a 20% validation set, the best  model on the validation set is chosen as the final RBM to encode values in the dataset as retrained on the whole training set.

In [13]:
def flatten_and_binarize(images):
    threshold=np.random.rand(*images.shape)
    return (images>threshold).reshape((-1,28*28)).astype(np.int32)

In [14]:
from sklearn.model_selection import ParameterGrid, train_test_split

random_state=42

np.random.seed(random_state)

rbm=RBM(28*28,50)

grid=ParameterGrid({
    "hidden_units":[50,100,200],
    "learning_rate":[0.001,0.01,0.1],
})

training=flatten_and_binarize(tr_images)
X_train,X_test,y_train,y_test=train_test_split(training,tr_labels,test_size=0.2,random_state=random_state,stratify=tr_labels)
best_loss=+np.inf
best_rbm=None
for params in grid:
    print(params)
    rbm=RBM(28*28,params['hidden_units'])
    #initialize visible bias with suggested values
    visible_active_prob=np.mean(X_train,axis=0)
    maxv=max(rbm.visible_bias[np.isfinite(rbm.visible_bias)])
    minv=min(rbm.visible_bias[np.isfinite(rbm.visible_bias)])
    rbm.visible_bias=np.log(visible_active_prob/(1-visible_active_prob))
    rbm.visible_bias=np.clip(rbm.visible_bias,minv,maxv)
    
    rbm.train(X_train,
            eta=params['learning_rate'],
            epochs=10,
            batch_size=10
            )
    recon_act=rbm.reconstruct(X_test)
    loss=np.mean((X_test-recon_act)**2)
    print(f"validation set reconstruction error {loss}")
    if loss < best_loss:
        best_loss=loss
        best_params=params
        best_rbm=rbm

buildinig a RBM with 784 visible units and 50 hidden units
{'hidden_units': 50, 'learning_rate': 0.001}
buildinig a RBM with 784 visible units and 50 hidden units
training over 48000 samples with 784 features 
epochs=10	 batch size=10	 learning rate=0.001


C:\Users\mirda\AppData\Local\Temp\ipykernel_22312\3213545003.py:25: RuntimeWarning: divide by zero encountered in log
  rbm.visible_bias=np.log(visible_active_prob/(1-visible_active_prob))


epoch no.1 reconstruction error: 0.19294637542517007
epoch no.2 reconstruction error: 0.16376270195578232
epoch no.3 reconstruction error: 0.14713847257653062
epoch no.4 reconstruction error: 0.13762178465136055
epoch no.5 reconstruction error: 0.13135615965136055
epoch no.6 reconstruction error: 0.1265911724064626
epoch no.7 reconstruction error: 0.12242421343537414
epoch no.8 reconstruction error: 0.11903063881802721
epoch no.9 reconstruction error: 0.11609271364795919
epoch no.10 reconstruction error: 0.11349782100340136
validation set reconstruction error 0.11312127976190477
{'hidden_units': 50, 'learning_rate': 0.01}
buildinig a RBM with 784 visible units and 50 hidden units
training over 48000 samples with 784 features 
epochs=10	 batch size=10	 learning rate=0.01
epoch no.1 reconstruction error: 0.113707137542517
epoch no.2 reconstruction error: 0.09929474914965987
epoch no.3 reconstruction error: 0.09257315582482993
epoch no.4 reconstruction error: 0.08861176658163265
epoch no.

Final retraining on the whole training set for the best model

In [15]:
test=flatten_and_binarize(ts_images)
final_rbm=RBM(28*28,best_params['hidden_units'])
#intialize visible bias with suggested values
visible_active_prob=np.mean(X_train,axis=0)
maxv=max(final_rbm.visible_bias[np.isfinite(final_rbm.visible_bias)])
minv=min(final_rbm.visible_bias[np.isfinite(final_rbm.visible_bias)])
final_rbm.visible_bias=np.log(visible_active_prob/(1-visible_active_prob))
final_rbm.visible_bias=np.clip(final_rbm.visible_bias,minv,maxv)

final_rbm.train(training,eta=best_params['learning_rate'],epochs=10,batch_size=10)

test_rec=final_rbm.reconstruct(test)
print(f"test reconstruction error {np.mean((test-test_rec)**2)}")

buildinig a RBM with 784 visible units and 200 hidden units
training over 60000 samples with 784 features 
epochs=10	 batch size=10	 learning rate=0.1


C:\Users\mirda\AppData\Local\Temp\ipykernel_22312\665517387.py:7: RuntimeWarning: divide by zero encountered in log
  final_rbm.visible_bias=np.log(visible_active_prob/(1-visible_active_prob))


epoch no.1 reconstruction error: 0.050544727891156466
epoch no.2 reconstruction error: 0.044902763605442174
epoch no.3 reconstruction error: 0.04290257227891157
epoch no.4 reconstruction error: 0.04124880952380952
epoch no.5 reconstruction error: 0.03955297619047619
epoch no.6 reconstruction error: 0.038997108843537416
epoch no.7 reconstruction error: 0.037762393707482994
epoch no.8 reconstruction error: 0.03736887755102041
epoch no.9 reconstruction error: 0.037178890306122446
epoch no.10 reconstruction error: 0.036231016156462584
test reconstruction error 0.03673545918367347


## encoding data
Now we first binarize the test data and then encode both training and test data using the hidden activations of the RBM.

In [16]:

h_train=final_rbm.encode(training)
h_test=final_rbm.encode(test)

60000 samples encoded with 200 hidden units
10000 samples encoded with 200 hidden units


# using the encodings

Now we can use the encoding of both training and test set to train a logistic regression to recognize the MNIST digits.
The logistic regression

In [17]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignore convergence warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [18]:
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import  train_test_split
from sklearn.linear_model import LogisticRegression

#make a model selection of the logistic regression
# Split the data into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(h_train,tr_labels,test_size=0.2,random_state=random_state,stratify=tr_labels)
# Define the parameter grid for grid search
param_grid = {'C': [0.1, 1, 10], 'penalty': ['l2'],'max_iter':[100,200,300]}
#param grid fire the linear SVM
best_fscore=-np.inf
best_params=None
for params in ParameterGrid(param_grid):
    model = LogisticRegression(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    fscore = f1_score(y_test, y_pred,average='micro')
    if fscore > best_fscore:
        best_fscore = fscore
        best_params = params
    print(f"Accuracy: {fscore} with parameters: {params}")
# Train the best model
print(f"best parameters {best_params}")
model = LogisticRegression(**best_params)
model.fit(h_train, tr_labels)
# Evaluate the model
y_pred = model.predict(h_test)
print(f"f1-score on test set: {f1_score(ts_labels, y_pred,average='micro')}")


Accuracy: 0.93125 with parameters: {'C': 0.1, 'max_iter': 100, 'penalty': 'l2'}
Accuracy: 0.93125 with parameters: {'C': 0.1, 'max_iter': 200, 'penalty': 'l2'}
Accuracy: 0.93125 with parameters: {'C': 0.1, 'max_iter': 300, 'penalty': 'l2'}
Accuracy: 0.9315833333333333 with parameters: {'C': 1, 'max_iter': 100, 'penalty': 'l2'}
Accuracy: 0.9315833333333333 with parameters: {'C': 1, 'max_iter': 200, 'penalty': 'l2'}
Accuracy: 0.9315833333333333 with parameters: {'C': 1, 'max_iter': 300, 'penalty': 'l2'}
Accuracy: 0.9304166666666667 with parameters: {'C': 10, 'max_iter': 100, 'penalty': 'l2'}
Accuracy: 0.93075 with parameters: {'C': 10, 'max_iter': 200, 'penalty': 'l2'}
Accuracy: 0.93075 with parameters: {'C': 10, 'max_iter': 300, 'penalty': 'l2'}
best parameters {'C': 1, 'max_iter': 100, 'penalty': 'l2'}
f1-score on test set: 0.9365


# Results and final considerations

In the first part the RBM, selected through a model selection, manages to achieve 0.03 of reconstrunction error on the test set showing a small loss of information during the reconstruction using 200 hidden units a batch size of 10 during training and a learning rate of 0.1. 
For the second part the logistic regression selected on a model selection using th same split used for the selected RBM manages to get a 0.93 of f1-score using microaveraging.

RBMs are good model that have the capability of learning representations from data, they offer a nice dimensionality reduction that lets use simpler models to achieve good results. However RBMs can be costly to train when we have a lot of data and we use a lot of hidden activations the trainig time increases significantly, also they offer little interpretation for the learned features.